### VGG 논문 구현 with CIFAR10
https://cryptosalamander.tistory.com/158   
이 친구 resnet도 있음.   
내일 출근(10/24)해서 전체코드 한번 싹 훑고   
resnet하고 MobileNet 찾아 보고 하자.   
하루에 네트워크 하나씩 하면 될 듯?!   
    
       

-----
2022/10/24 발견한 것.    
랜덤시드 고정 안해놔서 copy후 retrain 시키겠음.    

In [1]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from VGG import VGG11, VGG13, VGG16, VGG19
import os
import torchvision.models as models

''' random seed fixed'''
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [2]:
# Simple Learning Rate Scheduler
def lr_scheduler(optimizer, epoch):
    lr = learning_rate
    if epoch >= 30:
        lr /= 2
    if epoch >= 60:
        lr /= 2
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Xavier         
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform(m.weight)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='/home/sldev1/Project/hyeongeun_test/data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='/home/sldev1/Project/hyeongeun_test/data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = 'cuda'
model = VGG16(cifar=True)
# VGG11, VGG13, VGG16, VGG19 중에 택일하여 사용

In [5]:
model.apply(init_weights)
model = model.to(device)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  app.launch_new_instance()
/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [6]:
learning_rate = 0.05
num_epoch = 90
model_name = 'model.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

In [7]:
# Train
for epoch in range(num_epoch):
    print(f"====== { epoch+1} epoch of { num_epoch } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0
    # Train Phase
    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device) # batch[0]은 이미지 / batch[1]은 label
        optimizer.zero_grad()
        
        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)
        
        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()
        
        if step % 100 == 0 and step != 0:
            print(f"\n====== { step } Step of { len(train_loader) } ======")
            print(f"Train Acc : { correct / total_cnt }")
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")
            
    correct = 0
    total_cnt = 0
    
# Test Phase
    with torch.no_grad():
        model.eval()
        for step, batch in enumerate(test_loader):
            # input and target
            batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
            total_cnt += batch[1].size(0)
            logits = model(batch[0])
            valid_loss += loss_fn(logits, batch[1])
            _, predict = logits.max(1)
            correct += predict.eq(batch[1]).sum().item()
        valid_acc = correct / total_cnt
        print(f"\nValid Acc : { valid_acc }")    
        print(f"Valid Loss : { valid_loss / total_cnt }")

        if(valid_acc > best_acc):
            best_acc = valid_acc
            torch.save(model, model_name)
            print("Model Saved!")

====== 1 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.11478960396039604
Train Loss : 0.017640506848692894

====== 200 Step of 391 ======
Train Acc : 0.14369558457711443
Train Loss : 0.015766844153404236

====== 300 Step of 391 ======
Train Acc : 0.1583783222591362
Train Loss : 0.015507307834923267

Valid Acc : 0.2641
Valid Loss : 0.014975467696785927
Model Saved!
====== 2 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.25379022277227725
Train Loss : 0.014281193725764751

====== 200 Step of 391 ======
Train Acc : 0.2699782338308458
Train Loss : 0.013701274991035461

====== 300 Step of 391 ======
Train Acc : 0.28060112126245845
Train Loss : 0.014397591352462769

Valid Acc : 0.3664
Valid Loss : 0.012636153027415276
Model Saved!
====== 3 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.34282178217821785
Train Loss : 0.01264165062457323

====== 200 Step of 391 ======
Train Acc : 0.3615127487562189
Train Loss : 0.01393356267362833

====== 300


====== 300 Step of 391 ======
Train Acc : 0.8595826411960132
Train Loss : 0.0037171130534261465

Valid Acc : 0.8321
Valid Loss : 0.004234424326568842
====== 23 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.8661045792079208
Train Loss : 0.002007128670811653

====== 200 Step of 391 ======
Train Acc : 0.8629120024875622
Train Loss : 0.003111178521066904

====== 300 Step of 391 ======
Train Acc : 0.8624636627906976
Train Loss : 0.004205142613500357

Valid Acc : 0.8459
Valid Loss : 0.0037540735211223364
Model Saved!
====== 24 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.8639387376237624
Train Loss : 0.00303987436927855

====== 200 Step of 391 ======
Train Acc : 0.8670320273631841
Train Loss : 0.0025060216430574656

====== 300 Step of 391 ======
Train Acc : 0.8689264950166113
Train Loss : 0.0019152500899508595

Valid Acc : 0.839
Valid Loss : 0.004135717637836933
====== 25 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.8807240099009901
Tr


====== 200 Step of 391 ======
Train Acc : 0.9522699004975125
Train Loss : 0.0010891148122027516

====== 300 Step of 391 ======
Train Acc : 0.9514119601328903
Train Loss : 0.0013420400209724903

Valid Acc : 0.888
Valid Loss : 0.003128369338810444
====== 45 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.958694306930693
Train Loss : 0.001145943533629179

====== 200 Step of 391 ======
Train Acc : 0.9544465174129353
Train Loss : 0.0010041804052889347

====== 300 Step of 391 ======
Train Acc : 0.9537998338870431
Train Loss : 0.0007702873554080725

Valid Acc : 0.8903
Valid Loss : 0.0033587843645364046
Model Saved!
====== 46 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.9550587871287128
Train Loss : 0.0010716206161305308

====== 200 Step of 391 ======
Train Acc : 0.9521532960199005
Train Loss : 0.0006208753329701722

====== 300 Step of 391 ======
Train Acc : 0.9528394933554817
Train Loss : 0.000928973313421011

Valid Acc : 0.8803
Valid Loss : 0.0034087505191


====== 100 Step of 391 ======
Train Acc : 0.9853032178217822
Train Loss : 9.249378490494564e-05

====== 200 Step of 391 ======
Train Acc : 0.986396144278607
Train Loss : 0.000244845257839188

====== 300 Step of 391 ======
Train Acc : 0.9867369186046512
Train Loss : 0.00030434137443080544

Valid Acc : 0.8995
Valid Loss : 0.0036424959544092417
====== 67 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.987082301980198
Train Loss : 0.0008009917801246047

====== 200 Step of 391 ======
Train Acc : 0.9867848258706468
Train Loss : 0.00025984010426327586

====== 300 Step of 391 ======
Train Acc : 0.9871522009966778
Train Loss : 0.00024461859720759094

Valid Acc : 0.9002
Valid Loss : 0.003520425409078598
====== 68 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.9854579207920792
Train Loss : 0.0009675898472778499

====== 200 Step of 391 ======
Train Acc : 0.9860851990049752
Train Loss : 0.00015739485388621688

====== 300 Step of 391 ======
Train Acc : 0.98684073920


Valid Acc : 0.8982
Valid Loss : 0.004018573090434074
====== 88 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.9883199257425742
Train Loss : 4.069239366799593e-05

====== 200 Step of 391 ======
Train Acc : 0.988339552238806
Train Loss : 0.0001073273888323456

====== 300 Step of 391 ======
Train Acc : 0.9881904069767442
Train Loss : 0.0003774805227294564

Valid Acc : 0.9012
Valid Loss : 0.003819122677668929
====== 89 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.9934251237623762
Train Loss : 0.0001073659150279127

====== 200 Step of 391 ======
Train Acc : 0.9923429726368159
Train Loss : 0.00016942372894845903

====== 300 Step of 391 ======
Train Acc : 0.9906042358803987
Train Loss : 0.0003396897518541664

Valid Acc : 0.9022
Valid Loss : 0.0036991098895668983
====== 90 epoch of 90 ======

====== 100 Step of 391 ======
Train Acc : 0.9897122524752475
Train Loss : 0.000381868943804875

====== 200 Step of 391 ======
Train Acc : 0.990127487562189
Train Loss 